In [ ]:
!pip install bitsandbytes accelerate transformers torch -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.7 MB/s eta 0:00:00


# 한국어<->SQL 데이터셋
- 한국어가 SQL로 변환된 데이터셋 로드
- 데이터셋 기반으로 프롬프트 생성

In [ ]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

import pandas as pd

splits = {'train': 'clean/train.csv', 'test': 'test.csv'}
dataset = pd.read_csv("hf://datasets/shangrilar/ko_text2sql/" + splits["train"])

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  dataset.loc[idx, 'prompt'] = prompt

# 추론 파이프라인 생성
- 모델 및 토크나이저 가져오기
- 추론 파이프라인 생성

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "shangrilar/yi-ko-6b-text2sql"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Device set to use cuda:0


# 성능 최적화: 배치 크기에 따른 추론 시간 확인
- 배치 크기 조절

In [ ]:
import time
import numpy as np
for batch_size in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-a0a0b64c2d57>", line 5, in <cell line: 0>
    hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_generation.py", line 302, in __call__
    return super().__call__(text_inputs, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 1412, in __call__
    outputs = list(final_iterator)
              ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/pt_utils.py", line 124, in __next__
    item = next(self.iterator)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/pt_utils.py", line 12

TypeError: object of type 'NoneType' has no len()

In [ ]:
prompts = dataset['prompt'].tolist()[:128]  # 128~256까지 무난
batch_size = 32  # A100에서는 16~32도 잘 돌아감
#20일때 36.49
#32일때 25.43초
start_time = time.time()
outputs = hf_pipeline(prompts, max_new_tokens=128, batch_size=batch_size)
print(f"🔥 총 실행 시간: {time.time() - start_time:.2f}초")

# 결과 일부 확인
for i, output in enumerate(outputs[:3]):
    print(f"\n📝 Prompt #{i+1} 결과:\n{output[0]['generated_text']}\n")


🔥 총 실행 시간: 25.43초

📝 Prompt #1 결과:
당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);

### Question:
모든 플레이어 정보를 조회해 줘

### SQL:
SELECT * FROM players;


📝 Prompt #2 결과:
당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);

### Question:
모든 사용자의 아이디와 이메일을 보여줘

### SQL:
SELECT player_id, email FROM players;


📝 Prompt #3 결과:
당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY

# vLLM 성능 테스트
- vLLM:  대규모 언어 모델(LLM) 추론 및 서비스 제공을 위한 오픈소스 라이브러리
- max_num_seqs: KV 캐시에 저장할 최대 토큰 수(페이지 어텐션 처리)

In [ ]:
!pip install vllm transformers typing_extensions   -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7

In [ ]:
!pip install bitsandbytes accelerate transformers torch vllm transformers typing_extensions pyzmq fastai cudf-cu12 --upgrade -qqq

In [ ]:
# vllm과 pyzmq 라이브러리를 제거합니다.
!pip uninstall vllm pyzmq -y
# vllm과 pyzmq 라이브러리를 최신 버전으로 다시 설치합니다.
!pip install vllm pyzmq --upgrade -qqq

Found existing installation: vllm 0.9.0.1
Uninstalling vllm-0.9.0.1:
  Successfully uninstalled vllm-0.9.0.1
Found existing installation: pyzmq 26.4.0
Uninstalling pyzmq-26.4.0:
  Successfully uninstalled pyzmq-26.4.0


In [ ]:
import torch
import time
from vllm import LLM

# ✅ LLM 인스턴스를 만들 때 max_num_seqs를 설정
max_num_seqs = 20
model_id = "shangrilar/yi-ko-6b-text2sql"

llm = LLM(
    model=model_id,
    dtype=torch.float16,
    max_model_len=1024,
    max_num_seqs=20
)

sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
outputs = llm.generate(dataset['prompt'].tolist()[:128], sampling_params)
print(f'{max_num_seqs}: {time.time() - start_time}')

INFO 06-04 23:55:01 [config.py:793] This model supports multiple tasks: {'reward', 'embed', 'classify', 'score', 'generate'}. Defaulting to 'generate'.
INFO 06-04 23:55:01 [config.py:2118] Chunked prefill is enabled with max_num_batched_tokens=8192.


ValueError: Cannot cast <zmq.Socket(zmq.ROUTER) at 0x7f2e2c434d70> to int